In [1]:
from astropy.io import fits, ascii
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import rafias_lib as rl
import pdb, glob
%matplotlib inline

# General

In [2]:
#coefficient files
a1 = fits.open('/usr/local/nircamsuite/cal/Linearity/NRCA1_17004_LinearityCoeff_ADU0_2016-05-14.fits')
a1_coeff = a1[1].data

b4 = fits.open('/usr/local/nircamsuite/cal/Linearity/NRCB4_17047_LinearityCoeff_ADU0_2016-05-20.fits')
b4_coeff = b4[1].data

In [3]:
def correction(flux,corr_factor):
    """Correction function"""
    return np.polynomial.polynomial.polyval(flux, corr_factor)

**Naming Convention:**
+ First letter &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- test type &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: s = sub, s6 = sub640, f = full1
+ Second & Third &nbsp;- detector side &nbsp;&nbsp;&nbsp;: a1/b4
+ After dash (1st) &nbsp;- correction type : m = mmm, p = mpm
+ After dash (2nd) - file type &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: s = slp, r = red
+ After dash (3rd) &nbsp;- object type &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: f = file, s = stdev, t = ts, c = cen, i = image

In [4]:
#mmm red files
sa1_mrf, sb4_mrf, s6a1_mrf, s6b4_mrf, fa1_mrf, fb4_mrf = rl.fname_generator(['SUB', 'SUB640', 'FULL1'], 0)

#mmm slp files
#sa1_msf, sb4_msf, s6a1_msf, s6b4_msf, fa1_msf, fb4_msf  = rl.fname_generator(['SUB', 'SUB640', 'FULL1'], 6)

#mpm red files
sa1_prf, sb4_prf, s6a1_prf, s6b4_prf, fa1_prf, fb4_prf = rl.fname_generator(['SUB', 'SUB640', 'FULL1'], 0, 'MPM')

#mpm red files
# sa1_psf, sb4_psf, s6a1_psf, s6b4_psf, fa1_psf, fb4_psf = rl.fname_generator(['SUB', 'SUB640', 'FULL1'], 6, 'MPM')

In [5]:
cen_sa1, cen_sb4, cen_s6a1, cen_s6b4, cen_fa1, cen_fb4 = np.load('centers.npy')[:6]

In [6]:
def test_pixel(fname, pixels, stdev, coeff, plane, just_image = False):
    hdu = fits.open(fname)
    header = hdu[0].header
    image = hdu[0].data
    hdu.close()
    
    if just_image == True:
        %matplotlib notebook
        plt.imshow(image[plane])
        return image
    else:
        ori, cor, dev = np.array([]), np.array([]), np.array([])
        for p in pixels:
            x, y = p
            px1 = image[plane][y,x]
            px2 = px1*(1.+(stdev*1e-6))

            corr_x, corr_y = [int(x+header['COLCORNR']), int(y+header['ROWCORNR'])]
            corr_px1 = correction(px1, coeff[:,corr_y,corr_x])*px1
            corr_px2 = correction(px2, coeff[:,corr_y,corr_x])*px2

            orig_stdev = ((px2/px1) -1.)*1e6
            corr_stdev = ((corr_px2/corr_px1) -1.)*1e6
            deviation = ((orig_stdev-corr_stdev)/orig_stdev)*100.
            
            ori = np.append(ori, orig_stdev)
            cor = np.append(cor, corr_stdev)
            dev = np.append(dev, deviation)
        
        return ori, cor, dev 

In [7]:
def all_pixel(fname, cenx, ceny, stdev, coeff, plane, just_image = False, diff_plane = False, last_plane = None):
    
    hdu    = fits.open(fname)
    header = hdu[0].header
    image  = hdu[0].data
    hdu.close()
    
    if diff_plane == True:
        im1 = image[last_plane] - image[plane]
    else:
        im1 = image[plane]
    im2 = im1*(1.+(stdev*1e-6))
    
    x, y               = np.arange(0, im1.shape[0]), np.arange(0,im1.shape[1])
    col, row           = int(header['COLCORNR']) - 1, int(header['ROWCORNR']) - 1
    corr_x, corr_y     = (x + col), (y + row)
    corr_im1, corr_im2 = np.zeros([im1.shape[0],im1.shape[1]]), np.zeros([im1.shape[0],im1.shape[1]])
    
    for cy in corr_y:
        for cx in corr_x:
            corr_im1[cy-row][cx-col] = correction(im1[cy-row][cx-col], coeff[:, cy, cx])*im1[cy-row][cx-col]
            corr_im2[cy-row][cx-col] = correction(im2[cy-row][cx-col], coeff[:, cy, cx])*im2[cy-row][cx-col]
    
    mask       = np.isnan(im1) == True
    corr_mask  = np.isnan(corr_im1) == True
    flux1      = rl.photometry(im1, [cenx], [ceny], mask, rad = 70)[0]
    flux2      = rl.photometry(im2, [cenx], [ceny], mask, rad = 70)[0]
    corr_flux1 = rl.photometry(corr_im1, [cenx], [ceny], corr_mask, rad = 70)[0]
    corr_flux2 = rl.photometry(corr_im2, [cenx], [ceny], corr_mask, rad = 70)[0]
    orig_stdev = ((flux2/flux1) -1.)*1e6
    corr_stdev = ((corr_flux2/corr_flux1) -1.)*1e6
    deviation  = ((orig_stdev-corr_stdev)/orig_stdev)*100.


    return orig_stdev, corr_stdev, deviation 

# Testing A Few Pixels

## Sub

#### a1

In [8]:
sa1_mri = test_pixel(sa1_mrf[105], 0, 0, 0, 10, just_image = True)

<IPython.core.display.Javascript object>

In [9]:
sa1_pix = [(150, 185), (203, 147), (186, 135), (155, 99), (166, 160)]

In [10]:
sa1_mrt = rl.time_series(cen_sa1[1], cen_sa1[2], sa1_mrf, r = 70, r_in = 72, r_out = 80, red = True)
sa1_mrs = (np.std(sa1_mrt['res_flux'])/np.median(sa1_mrt['res_flux']))*1e6
sa1_mrs

1579.7323532095625

In [11]:
sa1_prt = rl.time_series(cen_sa1[1], cen_sa1[2], sa1_prf, r = 70, r_in = 72, r_out = 80, red = True)
sa1_prs = (np.std(sa1_prt['res_flux'])/np.median(sa1_prt['res_flux']))*1e6
sa1_prs

1615.1369883376904

In [12]:
sa1_std, sa1_cor, sa1_dev = test_pixel(sa1_mrf[105], sa1_pix, sa1_mrs, a1_coeff, 10)
print('pixel', ' ', 'original', '      ', 'corrected', '   ', 'deviation')
for i in range(5):
    print(sa1_pix[i], sa1_std[i], sa1_cor[i], sa1_dev[i])

('pixel', ' ', 'original', '      ', 'corrected', '   ', 'deviation')
((150, 185), 1579.7323532096286, 1626.1746269194966, -2.939882418405162)
((203, 147), 1579.7323532096286, 1615.4963533629996, -2.2639278154117228)
((186, 135), 1579.7323532096286, 1604.6495646859782, -1.5773058914520526)
((155, 99), 1579.7323532096286, 1606.0065625649322, -1.6632063844182741)
((166, 160), 1579.7323532096286, 1602.0500858171526, -1.4127540378710268)


In [13]:
res_by_test.groups[2]

NameError: name 'res_by_test' is not defined

#### b4

In [ ]:
sb4_mri = test_pixel(sb4_mrf[105], 0, 0, 0, 10, just_image = True)

In [ ]:
sb4_pix = [(135, 145), (190, 135), (172, 133), (181, 126), (159, 156)]

In [ ]:
sb4_mrt = rl.time_series(cen_sb4[1], cen_sb4[2], sb4_mrf[:-1], r = 70, r_in = 72, r_out = 80, red = True)
sb4_mrs = (np.std(sb4_mrt['res_flux'])/np.median(sb4_mrt['res_flux']))*1e6
sb4_mrs

In [ ]:
sb4_prt = rl.time_series(cen_sb4[1], cen_sb4[2], sb4_prf[:-1], r = 70, r_in = 72, r_out = 80, red = True)
sb4_prs = (np.std(sb4_prt['res_flux'])/np.median(sb4_prt['res_flux']))*1e6
sb4_prs

In [ ]:
sb4_std, sb4_cor, sb4_dev = test_pixel(sb4_mrf[105], sb4_pix, sb4_mrs, b4_coeff, 10)
print('pixel', ' ', 'original', '      ', 'corrected', '   ', 'deviation')
for i in range(5):
    print(sb4_pix[i], sb4_std[i], sb4_cor[i], sb4_dev[i])

In [ ]:
res_by_test.groups[3]

## Sub640

#### a1

In [ ]:
s6a1_mri = test_pixel(s6a1_mrf[27], 0, 0, 0, 3, just_image = True)

In [ ]:
s6a1_pix = [(335, 335), (291, 326), (349, 290), (365, 370), (327, 320)]

In [ ]:
s6a1_mrt = rl.time_series(cen_s6a1[1], cen_s6a1[2], s6a1_mrf, r = 70, r_in = 72, r_out = 80, red = True)
s6a1_mrs = (np.std(s6a1_mrt['res_flux'])/np.median(s6a1_mrt['res_flux']))*1e6
s6a1_mrs

In [ ]:
s6a1_prt = rl.time_series(cen_s6a1[1], cen_s6a1[2], s6a1_prf, r = 70, r_in = 72, r_out = 80, red = True)
s6a1_prs = (np.std(s6a1_prt['res_flux'])/np.median(s6a1_prt['res_flux']))*1e6
s6a1_prs

In [ ]:
s6a1_std, s6a1_cor, s6a1_dev = test_pixel(s6a1_mrf[27], s6a1_pix, s6a1_mrs, a1_coeff, 3)
print('pixel', ' ', 'original', '      ', 'corrected', '   ', 'deviation')
for i in range(5):
    print(s6a1_pix[i], s6a1_std[i], s6a1_cor[i], s6a1_dev[i])

In [ ]:
res_by_test.groups[4]

#### b4

In [ ]:
s6b4_mri = test_pixel(s6b4_mrf[27], 0, 0, 0, 3, just_image = True)

In [ ]:
s6b4_pix = [(295, 305), (347, 320), (329, 364), (356, 270), (319, 316)]

In [ ]:
s6b4_mrt = rl.time_series(cen_s6b4[1], cen_s6b4[2], s6b4_mrf, r = 70, r_in = 72, r_out = 80, red = True)
s6b4_mrs = (np.std(s6b4_mrt['res_flux'])/np.median(s6b4_mrt['res_flux']))*1e6
s6b4_mrs

In [ ]:
s6b4_prt = rl.time_series(cen_s6b4[1], cen_s6b4[2], s6b4_prf, r = 70, r_in = 72, r_out = 80, red = True)
s6b4_prs = (np.std(s6b4_prt['res_flux'])/np.median(s6b4_prt['res_flux']))*1e6
s6b4_prs

In [ ]:
s6b4_std, s6b4_cor, s6b4_dev = test_pixel(s6b4_mrf[27], s6b4_pix, s6b4_mrs, b4_coeff, 3)
print('pixel', ' ', 'original', '      ', 'corrected', '   ', 'deviation')
for i in range(5):
    print(s6b4_pix[i], s6b4_std[i], s6b4_cor[i], s6b4_dev[i])

In [ ]:
res_by_test.groups[5]

## FULL1

#### a1

In [ ]:
fa1_mri = test_pixel(fa1_mrf[27], 0, 0, 0, 1, just_image = True)

In [ ]:
fa1_pix = [(1390, 1060), (1447, 1054), (1413, 1088), (1390, 975), (1405, 1037)]

In [ ]:
fa1_mrt = rl.time_series(cen_fa1[1], cen_fa1[2], fa1_mrf, r = 70, r_in = 72, r_out = 80, red = True)
fa1_mrs = (np.std(fa1_mrt['res_flux'])/np.median(fa1_mrt['res_flux']))*1e6
fa1_mrs

In [ ]:
fa1_prt = rl.time_series(cen_fa1[1], cen_fa1[2], fa1_prf, r = 70, r_in = 72, r_out = 80, red = True)
fa1_prs = (np.std(fa1_prt['res_flux'])/np.median(fa1_prt['res_flux']))*1e6
fa1_prs

In [ ]:
fa1_std, fa1_cor, fa1_dev = test_pixel(fa1_mrf[27], fa1_pix, fa1_mrs, a1_coeff, 1)
print('pixel', ' ', 'original', '      ', 'corrected', '   ', 'deviation')
for i in range(5):
    print(fa1_pix[i], fa1_std[i], fa1_cor[i], fa1_dev[i])

In [ ]:
res_by_test.groups[0]

#### b4

In [ ]:
fb4_mri = test_pixel(fb4_mrf[27], 0, 0, 0, 1, just_image = True)

In [ ]:
fb4_pix = [(853, 808), (844, 851), (840, 796), (802, 789), (828, 821)]

In [ ]:
fb4_mrt = rl.time_series(cen_fb4[1], cen_fb4[2], fb4_mrf, r = 70, r_in = 72, r_out = 80, red = True)
fb4_mrs = (np.std(fb4_mrt['res_flux'])/np.median(fb4_mrt['res_flux']))*1e6
fb4_mrs

In [ ]:
fb4_prt = rl.time_series(cen_fb4[1], cen_fb4[2], fb4_prf, r = 70, r_in = 72, r_out = 80, red = True)
fb4_prs = (np.std(fb4_prt['res_flux'])/np.median(fb4_prt['res_flux']))*1e6
fb4_prs

In [ ]:
fb4_std, fb4_cor, fb4_dev = test_pixel(fb4_mrf[27], fb4_pix, fb4_mrs, b4_coeff, 1)
print('pixel', ' ', 'original', '      ', 'corrected', '   ', 'deviation')
for i in range(5):
    print(fb4_pix[i], fb4_std[i], fb4_cor[i], fb4_dev[i])

In [ ]:
res_by_test.groups[1]

## Result

In [ ]:
tests = ['Sub a1', 'Sub b4', 'Sub640 a1', 'Sub640 b4', 'Full1 a1', 'Full1 b4']
test_short = ['sa1', 'sb4', 's6a1', 's6b4', 'fa1', 'fb4']
result = Table(names= ('Test Name', 'Pixel','Pixel value','MMM', 'MPM', 'Corrected MMM', 'Deviation (%)'), 
               dtype = ('S10', np.dtype(tuple), 'f8', 'f8', 'f8', 'f8', 'f8'))
for i, t in enumerate(test_short):
    test_img = globals()['%s_mri' % t]
    test_pix = globals()['%s_pix' % t]
    pix_value = []
    
    for pix in test_pix:
        x,y = pix
        z = 10 if i<2 else 3 if i<4 else 1
        pix_value.append(test_img[z, y, x])
    ind = np.argsort(pix_value)
    
    mmm = globals()['%s_mrs' % t]
    mpm = globals()['%s_prs' % t]
    pix = np.array(test_pix)[ind]
    cor = globals()['%s_cor' % t][ind]
    dev = globals()['%s_dev' % t][ind]*(-1)
#     pdb.set_trace()
    
    for pv, p, c, d in zip(np.sort(pix_value), pix, cor, dev):
        result.add_row([tests[i], p, pv, mmm, mpm, c, d])
res_by_test = result.group_by('Test Name')
res_by_test

# More Accurate: All Pixels 

## Sub

#### a1

Last frame

In [ ]:
sa1_lf_ori, sa1_lf_cor, sa1_lf_dev = all_pixel(sa1_mrf[105], 166, 160, sa1_mrs, a1_coeff, 19)
sa1_lf_ori, sa1_lf_cor, sa1_lf_dev

 First frame

In [ ]:
sa1_ff_ori, sa1_ff_cor, sa1_ff_dev = all_pixel(sa1_mrf[105], 166, 160, sa1_mrs, a1_coeff, 0)
sa1_ff_ori, sa1_ff_cor, sa1_ff_dev

Last - First

In [ ]:
sa1_df_ori, sa1_df_cor, sa1_df_dev = all_pixel(sa1_mrf[105], 166, 160, sa1_mrs, a1_coeff, 0, diff_plane = True,
                                               last_plane = 19)
sa1_df_ori, sa1_df_cor, sa1_df_dev

### b4

 Last frame

In [ ]:
sb4_lf_ori, sb4_lf_cor, sb4_lf_dev = all_pixel(sb4_mrf[105], 159, 156, sb4_mrs, b4_coeff, 19)
sb4_lf_ori, sb4_lf_cor, sb4_lf_dev

First frame

In [ ]:
sb4_ff_ori, sb4_ff_cor, sb4_ff_dev = all_pixel(sb4_mrf[105], 159, 156, sb4_mrs, b4_coeff, 0)
sb4_ff_ori, sb4_ff_cor, sb4_ff_dev

Last - First

In [ ]:
sb4_df_ori, sb4_df_cor, sb4_df_dev = all_pixel(sb4_mrf[105], 159, 156, sb4_mrs, b4_coeff, 0, diff_plane = True,
                                               last_plane = 19)
sb4_df_ori, sb4_df_cor, sb4_df_dev

## Sub640

### a1

 Last frame

In [ ]:
s6a1_lf_ori, s6a1_lf_cor, s6a1_lf_dev = all_pixel(s6a1_mrf[27], 327, 320, s6a1_mrs, a1_coeff, 5)
s6a1_lf_ori, s6a1_lf_cor, s6a1_lf_dev

First frame

In [ ]:
s6a1_ff_ori, s6a1_ff_cor, s6a1_ff_dev = all_pixel(s6a1_mrf[27], 327, 320, s6a1_mrs, a1_coeff, 0)
s6a1_ff_ori, s6a1_ff_cor, s6a1_ff_dev

Last - First

In [ ]:
s6a1_df_ori, s6a1_df_cor, s6a1_df_dev = all_pixel(s6a1_mrf[27], 327, 320, s6a1_mrs, a1_coeff, 0, diff_plane = True, 
                                                  last_plane = 5)
s6a1_df_ori, s6a1_df_cor, s6a1_df_dev

### b4

 Last frame

In [ ]:
s6b4_lf_ori, s6b4_lf_cor, s6b4_lf_dev = all_pixel(s6b4_mrf[27], 319, 316, s6b4_mrs, b4_coeff, 5)
s6b4_lf_ori, s6b4_lf_cor, s6b4_lf_dev

First frame

In [ ]:
s6b4_ff_ori, s6b4_ff_cor, s6b4_ff_dev = all_pixel(s6b4_mrf[27], 319, 316, s6b4_mrs, b4_coeff, 0)
s6b4_ff_ori, s6b4_ff_cor, s6b4_ff_dev

Last - First

In [ ]:
s6b4_df_ori, s6b4_df_cor, s6b4_df_dev = all_pixel(s6b4_mrf[27], 319, 316, s6b4_mrs, b4_coeff, 0, diff_plane = True, 
                                                  last_plane = 5)
s6b4_df_ori, s6b4_df_cor, s6b4_df_dev

## Full1

### a1

 Last frame

In [ ]:
fa1_lf_ori, fa1_lf_cor, fa1_lf_dev = all_pixel(fa1_mrf[27], 1405, 1037, fa1_mrs, a1_coeff, 1)
fa1_lf_ori, fa1_lf_cor, fa1_lf_dev

First frame

In [ ]:
fa1_ff_ori, fa1_ff_cor, fa1_ff_dev = all_pixel(fa1_mrf[27], 1405, 1037, fa1_mrs, a1_coeff, 0)
fa1_ff_ori, fa1_ff_cor, fa1_ff_dev

Last - First

In [ ]:
fa1_df_ori, fa1_df_cor, fa1_df_dev = all_pixel(fa1_mrf[27], 1405, 1037, fa1_mrs, a1_coeff, 0, diff_plane = True, 
                                               last_plane = 1)
fa1_df_ori, fa1_df_cor, fa1_df_dev

### b4

 Last frame

In [ ]:
fb4_lf_ori, fb4_lf_cor, fb4_lf_dev = all_pixel(fb4_mrf[27], 828, 821, fb4_mrs, b4_coeff, 1)
fb4_lf_ori, fb4_lf_cor, fb4_lf_dev

First frame

In [ ]:
fb4_ff_ori, fb4_ff_cor, fb4_ff_dev = all_pixel(fb4_mrf[27], 828, 821, fb4_mrs, b4_coeff, 0)
fb4_ff_ori, fb4_ff_cor, fb4_ff_dev

Last - First

In [ ]:
fb4_df_ori, fb4_df_cor, fb4_df_dev = all_pixel(fb4_mrf[27], 828, 821, fb4_mrs, b4_coeff, 0, diff_plane = True, 
                                               last_plane = 1)
fb4_df_ori, fb4_df_cor, fb4_df_dev

## Results

In [ ]:
test = ['Sub a1 (lf)', 'Sub a1 (ff)', 'Sub a1 (df)', 'Sub b4 (lf)', 'Sub b4 (ff)', 'Sub b4 (df)', 'Sub640 a1 (lf)',
        'Sub640 a1 (ff)', 'Sub640 a1 (df)', 'Sub640 b4 (lf)',  'Sub640 b4 (ff)', 'Sub640 b4 (df)', 'Full1 a1 (lf)',
        'Full1 a1 (ff)', 'Full1 a1 (df)', 'Full1 b4 (lf)', 'Full1 b4 (ff)', 'Full1 b4 (df)']

test_sh = ['sa1_lf', 'sa1_ff', 'sa1_df', 'sb4_lf', 'sb4_ff', 'sb4_df', 's6a1_lf', 's6a1_ff', 's6a1_df', 's6b4_lf',
             's6b4_ff', 's6b4_df', 'fa1_lf', 'fa1_ff', 'fa1_df', 'fb4_lf', 'fb4_ff', 'fb4_df']

result = Table(names= ('Test Name', 'Calculated MMM', 'Corrected MMM', 'Deviation (%)'), 
               dtype = ('S16', 'f8', 'f8', 'f8'))

for i, t in enumerate(test_sh):
    tname = test[i]
    ori = globals()['%s_ori' % t]
    cor = globals()['%s_cor' % t]
    dev = globals()['%s_dev' % t]*(-1)
    result.add_row([tname, ori, cor, dev])
result